In [24]:
import tensorflow as tf
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec


In [40]:
saved_model_path = 'ipl_model.h5'

try:
    # Load the SavedModel
    model = tf.keras.models.load_model(saved_model_path)
    print("Model loaded successfully!")
except FileNotFoundError:
    print(f"Error: File '{saved_model_path}' not found.")
except Exception as e:
    print(f"Error: An unexpected error occurred: {e}")


Model loaded successfully!


In [41]:
word2vec_model = Word2Vec.load('word2vec_model.bin')

In [71]:
def create_vector(list_of_comments):
    
    sentence_vectors = []
    # Iterate over each sentence in the corpus
    for sentence in list_of_comments:
        # Tokenize the sentence and convert to lowercase
        tokens = word_tokenize(sentence.lower())

        
        word_vectors = []
        # Iterate over each word in the sentence
        for token in tokens:
            try:
                # Retrieve the word vector from the Word2Vec model
                word_vector = word2vec_model.wv[token]
                
                # Append the word vector to the list of word vectors
                word_vectors.append(word_vector)
            except KeyError:
                # Handle out-of-vocabulary words
        
                pass
        # If there are word vectors for the sentence
        if word_vectors:
            # word found -> mean of the vector
            sentence_vector = np.mean(word_vectors, axis=0)
        else:
            # no word -> 0's
            sentence_vector = np.zeros(word2vec_model.vector_size)
        sentence_vectors.append(sentence_vector)
    
    # list -> np array
    sentence_vectors = np.array(sentence_vectors)
    return sentence_vectors

In [46]:
label_mapping = {8: 'Wicket',
 1: 'One Run',
 4: 'Four Runs',
 6: 'Six Runs',
 0: 'Zero Runs',
 2: 'Two Runs',
 7: 'Seven Runs',
 5: 'Five Runs',
 3: 'Three Runs'}

In [48]:
def predict_score(text):
    # Converting text to preprocessing
    text_vectorization = create_vector([text])
    # Predicting the given text
    predicted_value = model.predict(text_vectorization)
    print(label_mapping[predicted_value.argmax()])


In [69]:
text = " short of good length ball, pitching on leg stump, Rilee Rossouw gets the hang of the bounce and smashes the pull towards deep square leg for another full montyy! "

In [70]:
predict_score(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Six Runs
